In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np


In [ ]:
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies = pd.read_csv('movies.csv')

In [ ]:
# Prepare matrices
user_ids = ratings["userId"].unique()
movie_ids = ratings["movieId"].unique()
n_users = len(user_ids)
n_items = len(movie_ids)

user_map = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_map = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}


In [ ]:
R = np.zeros((n_users, n_items))
for _, row in ratings.iterrows():
    user_idx = user_map[row["userId"]]
    movie_idx = movie_map[row["movieId"]]
    R[user_idx, movie_idx] = row["rating"]

# Convert to PyTorch tensors
R_tensor = torch.tensor(R, dtype=torch.float32)

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# Hyperparameters
n_factors = 10  # Number of latent factors
alpha = 0.01    # Learning rate
lambda_reg = 0.1  # Regularization strength
n_epochs = 1000  # Number of epochs


In [ ]:
# Model Definition
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_items, n_factors):
        super(MatrixFactorization, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)  # User latent features
        self.item_factors = nn.Embedding(n_items, n_factors)  # Item latent features

    def forward(self, user_idx, item_idx):
        user_latent = self.user_factors(user_idx)
        item_latent = self.item_factors(item_idx)
        return (user_latent * item_latent).sum(dim=1)


In [ ]:
# Initialize model and move to GPU
model = MatrixFactorization(n_users, n_items, n_factors).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=alpha)


In [ ]:
# Prepare data and move to GPU
user_indices, item_indices = R.nonzero()
ratings = torch.tensor(R[user_indices, item_indices], dtype=torch.float32).to(device)
user_indices = torch.tensor(user_indices, dtype=torch.long).to(device)
item_indices = torch.tensor(item_indices, dtype=torch.long).to(device)


In [ ]:
# Training loop with GPU support
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()

    # Predict ratings and compute loss
    predictions = model(user_indices, item_indices)
    loss = loss_fn(predictions, ratings)

    # Add regularization
    reg_loss = lambda_reg * (model.user_factors.weight.norm(2) + model.item_factors.weight.norm(2))
    total_loss = loss + reg_loss

    # Backpropagation
    total_loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.4f}")


Epoch 0, Loss: 62.2244
Epoch 50, Loss: 40.3733
Epoch 100, Loss: 26.1636
Epoch 150, Loss: 18.1215
Epoch 200, Loss: 14.6853
Epoch 250, Loss: 13.2494
Epoch 300, Loss: 12.7762
Epoch 350, Loss: 12.6170
Epoch 400, Loss: 12.5483
Epoch 450, Loss: 12.5117
Epoch 500, Loss: 12.4898
Epoch 550, Loss: 12.4757
Epoch 600, Loss: 12.4664
Epoch 650, Loss: 12.4599
Epoch 700, Loss: 12.4553
Epoch 750, Loss: 12.4520
Epoch 800, Loss: 12.4495
Epoch 850, Loss: 12.4477
Epoch 900, Loss: 12.4463
Epoch 950, Loss: 12.4452


In [ ]:
# Extract latent features
P = model.user_factors.weight.detach().cpu().numpy()  # User latent matrix
Q = model.item_factors.weight.detach().cpu().numpy()  # Item latent matrix


In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [ ]:
# Fit KNN model on Q
knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=20, n_jobs=-1)
knn.fit(Q)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=20)

In [ ]:
def recommend_movies(movieId, movie_map, movie_map_reverse, knn_model, Q):
    """
    Recommend top 20 similar movies to the given movieId.

    Args:
        movieId (int): ID of the movie for which recommendations are needed.
        movie_map (dict): Dictionary mapping movieId to row index in Q.
        movie_map_reverse (dict): Dictionary mapping row index in Q to movieId.
        knn_model: Trained KNN model.
        Q (ndarray): Item latent feature matrix.

    Returns:
        list: List of (movieId, similarity score) tuples for top 20 similar movies.
    """
    if movieId not in movie_map:
        raise ValueError(f"MovieId {movieId} is not in the dataset.")

    movie_idx = movie_map[movieId]
    distances, indices = knn_model.kneighbors([Q[movie_idx]], n_neighbors=41)  # n_neighbors=21 to exclude the movie itself

    recommendations = []
    for dist, idx in zip(distances[0][1:], indices[0][1:]):  # Exclude the input movie itself
        similar_movieId = movie_map_reverse[idx]
        recommendations.append((similar_movieId, 1 - dist))  # 1 - dist converts cosine distance to similarity

    return recommendations


In [ ]:
# Create reverse map for mapping indices back to movieId
movie_map_reverse = {v: k for k, v in movie_map.items()}


In [ ]:
# Example usage
movie_id = 1  # Replace with a valid movieId from your dataset
top_recommendations = recommend_movies(movie_id, movie_map, movie_map_reverse, knn, Q)


In [ ]:
print(f"Top recommendations for MovieId {movie_id}:")
for rec_movie_id, score in top_recommendations:
    print(f"MovieId: {rec_movie_id}, Similarity: {score:.4f}")

Top recommendations for MovieId 1:
MovieId: 165, Similarity: 1.0000
MovieId: 454, Similarity: 1.0000
MovieId: 527, Similarity: 1.0000
MovieId: 434, Similarity: 0.9999
MovieId: 161, Similarity: 0.9999
MovieId: 349, Similarity: 0.9999
MovieId: 588, Similarity: 0.9999
MovieId: 736, Similarity: 0.9999
MovieId: 500, Similarity: 0.9999
MovieId: 2571, Similarity: 0.9999
MovieId: 592, Similarity: 0.9998
MovieId: 50, Similarity: 0.9998
MovieId: 377, Similarity: 0.9998
MovieId: 185, Similarity: 0.9998
MovieId: 367, Similarity: 0.9998
MovieId: 292, Similarity: 0.9997
MovieId: 648, Similarity: 0.9997
MovieId: 339, Similarity: 0.9997
MovieId: 316, Similarity: 0.9997
MovieId: 587, Similarity: 0.9997
MovieId: 260, Similarity: 0.9997
MovieId: 344, Similarity: 0.9997
MovieId: 364, Similarity: 0.9997
MovieId: 137, Similarity: 0.9996
MovieId: 590, Similarity: 0.9996
MovieId: 589, Similarity: 0.9996
MovieId: 231, Similarity: 0.9996
MovieId: 32, Similarity: 0.9995
MovieId: 595, Similarity: 0.9994
MovieId: 

In [ ]:
def print_movie_recommendations(top_recommendations, movies):
    """
    Print the top movie recommendations with similarity scores.

    Args:
        top_recommendations (list of tuples): List containing tuples of (movieId, similarity_score)
        movies (DataFrame): DataFrame containing movieId and title

    Returns:
        None
    """
    print(f"Top recommendations:")
    for rec_movie_id, score in top_recommendations:
        # Get the movie title from the movieId
        movie_title = movies[movies['movieId'] == rec_movie_id]['title'].values[0]
        print(f"{movie_title}, Similarity: {score:.4f}")


In [ ]:
def get_movie_id_from_title(title, movies):
    movie_id = movies[movies['title'] == title]['movieId'].values
    if len(movie_id) > 0:
        return movie_id[0]
    else:
        return None  # Return None if movie title not found


In [ ]:
title = "Mad Max: Fury Road (2015)"  # Replace with a movie title
movie_id = get_movie_id_from_title(title, movies)

In [ ]:
top_recommendations = recommend_movies(movie_id, movie_map, movie_map_reverse, knn, Q)


In [ ]:
print_movie_recommendations(top_recommendations=top_recommendations,movies=movies)

Top recommendations:
Selfish Giant, The (2013), Similarity: 1.0000
The Dark Tower (2017), Similarity: 1.0000
Mamma Mia: Here We Go Again! (2018), Similarity: 1.0000
Logan Lucky (2017), Similarity: 1.0000
Race to Witch Mountain (2009), Similarity: 1.0000
The Secret Life of Pets (2016), Similarity: 1.0000
The Mummy (2017), Similarity: 1.0000
Black Mirror, Similarity: 1.0000
Way Back, The (2010), Similarity: 1.0000
Game Night (2018), Similarity: 1.0000
Moana (2016), Similarity: 1.0000
Life of Pi (2012), Similarity: 1.0000
The Revenant (2015), Similarity: 1.0000
Oblivion (2013), Similarity: 1.0000
What If (2013), Similarity: 1.0000
The Hunger Games: Mockingjay - Part 1 (2014), Similarity: 1.0000
Beauty and the Beast (2017), Similarity: 1.0000
Black Mirror: White Christmas (2014), Similarity: 1.0000
Journey to the West: Conquering the Demons (Daai wa sai you chi Chui mo chun kei) (2013), Similarity: 1.0000
Alpha (2018), Similarity: 1.0000
Librarian: Quest for the Spear, The (2004), Similari

In [ ]:
import pickle

# List of variables to save
variables_to_save = {
    'P': P,
    'Q': Q,
    'movie_map_reverse': movie_map_reverse,
    'user_map': user_map,
    'knn_model': knn,
    'movie_ids': movie_ids,
    'user_ids': user_ids
}

# Save the variables to a file
with open('/content/model_variables.pkl', 'wb') as f:
    pickle.dump(variables_to_save, f)

# using gidserach cv for best prams

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Matrix Factorization Model
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_movies, n_factors):
        super(MatrixFactorization, self).__init__()
        # User and movie embeddings
        self.user_factors = nn.Embedding(num_users, n_factors)
        self.movie_factors = nn.Embedding(num_movies, n_factors)

    def forward(self, user, movie):
        # Compute the dot product of user and movie latent vectors
        return (self.user_factors(user) * self.movie_factors(movie)).sum(1)

# Helper function to train and evaluate MF with PyTorch
def train_and_evaluate_mf(model, train_data, test_data, alpha, lambda_reg, n_epochs):
    # Loss function (mean squared error)
    loss_fn = nn.MSELoss()

    # Optimizer (Stochastic Gradient Descent)
    optimizer = optim.SGD(model.parameters(), lr=alpha)

    # Move data to device (GPU or CPU)
    train_data = train_data.to(device)
    test_data = test_data.to(device)

    # Training loop
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()

        # Get the users, movies, and ratings from training data
        users = train_data[:, 0].long()  # Ensure it's Long tensor
        movies = train_data[:, 1].long()  # Ensure it's Long tensor
        ratings = train_data[:, 2].float()

        # Make predictions
        predictions = model(users, movies)

        # Compute loss
        loss = loss_fn(predictions, ratings)

        # Add regularization term
        loss += lambda_reg * (model.user_factors(users).norm(2).sum() + model.movie_factors(movies).norm(2).sum())

        # Backpropagation
        loss.backward()
        optimizer.step()

    # Evaluation on the test set
    model.eval()
    with torch.no_grad():
        users = test_data[:, 0].long()  # Ensure it's Long tensor
        movies = test_data[:, 1].long()  # Ensure it's Long tensor
        true_ratings = test_data[:, 2].float()

        # Make predictions
        predictions = model(users, movies)

        # Compute RMSE (use torch.sqrt and torch.mean)
        mse = torch.mean((predictions - true_ratings) ** 2)
        rmse = torch.sqrt(mse)

    return rmse.item()  # Convert tensor to Python float for easy display

# Grid Search Loop
def grid_search(ratings, n_factors_options, alpha_options, lambda_reg_options, n_epochs_options):
    # Prepare the dataset
    user_ids = ratings['userId'].unique()
    movie_ids = ratings['movieId'].unique()

    # Map user and movie ids to indices
    user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
    movie_to_idx = {movie: idx for idx, movie in enumerate(movie_ids)}

    # Encode userId and movieId as indices
    ratings['userId'] = ratings['userId'].map(user_to_idx)
    ratings['movieId'] = ratings['movieId'].map(movie_to_idx)

    # Train-test split
    train_data, test_data = train_test_split(ratings, test_size=0.2)

    # Convert to PyTorch tensors
    train_tensor = torch.tensor(train_data[['userId', 'movieId', 'rating']].values)
    test_tensor = torch.tensor(test_data[['userId', 'movieId', 'rating']].values)

    # Initialize variables to track best parameters
    best_rmse = float('inf')
    best_params = None

    # Grid search over hyperparameters
    for n_factors in n_factors_options:
        for alpha in alpha_options:
            for lambda_reg in lambda_reg_options:
                for n_epochs in n_epochs_options:
                    print(f"Training with n_factors={n_factors}, alpha={alpha}, lambda_reg={lambda_reg}, n_epochs={n_epochs}")

                    # Initialize model
                    model = MatrixFactorization(len(user_ids), len(movie_ids), n_factors).to(device)

                    # Train and evaluate the model
                    rmse = train_and_evaluate_mf(model, train_tensor, test_tensor, alpha, lambda_reg, n_epochs)
                    print(f"RMSE: {rmse}")

                    # Update the best model if current is better
                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_params = {
                            'n_factors': n_factors,
                            'alpha': alpha,
                            'lambda_reg': lambda_reg,
                            'n_epochs': n_epochs
                        }

    print(f"Best Parameters: {best_params}")
    print(f"Best RMSE: {best_rmse}")

# Define the grid of hyperparameters to search
n_factors_options = [10, 20, 30, 40, 50]
alpha_options = [0.001, 0.01, 0.1]
lambda_reg_options = [0.01, 0.1, 0.5]
n_epochs_options = [500, 1000, 1500]

# Load your dataset (ensure it contains userId, movieId, and rating)
ratings = pd.read_csv('ratings.csv')

# Perform Grid Search
grid_search(ratings, n_factors_options, alpha_options, lambda_reg_options, n_epochs_options)


Training with n_factors=10, alpha=0.001, lambda_reg=0.01, n_epochs=500
RMSE: 4.865091800689697
Training with n_factors=10, alpha=0.001, lambda_reg=0.01, n_epochs=1000
RMSE: 4.799771785736084
Training with n_factors=10, alpha=0.001, lambda_reg=0.01, n_epochs=1500
RMSE: 4.759719371795654
Training with n_factors=10, alpha=0.001, lambda_reg=0.1, n_epochs=500
RMSE: 4.694297790527344
Training with n_factors=10, alpha=0.001, lambda_reg=0.1, n_epochs=1000
RMSE: 4.60194206237793
Training with n_factors=10, alpha=0.001, lambda_reg=0.1, n_epochs=1500
RMSE: 4.681056499481201
Training with n_factors=10, alpha=0.001, lambda_reg=0.5, n_epochs=500
RMSE: 4.504985809326172
Training with n_factors=10, alpha=0.001, lambda_reg=0.5, n_epochs=1000
RMSE: 4.38636589050293
Training with n_factors=10, alpha=0.001, lambda_reg=0.5, n_epochs=1500
RMSE: 4.249305725097656
Training with n_factors=10, alpha=0.01, lambda_reg=0.01, n_epochs=500
RMSE: 4.594008445739746
Training with n_factors=10, alpha=0.01, lambda_reg=0.

In [ ]:
def get_best_P_and_Q(ratings, best_params):
    # Extract best hyperparameters
    n_factors = best_params['n_factors']
    alpha = best_params['alpha']
    lambda_reg = best_params['lambda_reg']
    n_epochs = best_params['n_epochs']

    # Prepare the dataset
    user_ids = ratings['userId'].unique()
    movie_ids = ratings['movieId'].unique()

    # Map user and movie ids to indices
    user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
    movie_to_idx = {movie: idx for idx, movie in enumerate(movie_ids)}

    # Encode userId and movieId as indices
    ratings['userId'] = ratings['userId'].map(user_to_idx)
    ratings['movieId'] = ratings['movieId'].map(movie_to_idx)

    # Train-test split
    train_data, test_data = train_test_split(ratings, test_size=0.2)

    # Convert to PyTorch tensors
    train_tensor = torch.tensor(train_data[['userId', 'movieId', 'rating']].values).to(device)
    test_tensor = torch.tensor(test_data[['userId', 'movieId', 'rating']].values).to(device)

    # Initialize the final model with the best hyperparameters
    model = MatrixFactorization(len(user_ids), len(movie_ids), n_factors).to(device)

    # Train the model with the best configuration
    _ = train_and_evaluate_mf(model, train_tensor, test_tensor, alpha, lambda_reg, n_epochs)

    # Get the P (user factors) and Q (movie factors) matrices from the trained model
    P = model.user_factors.weight.data.cpu().numpy()  # User latent factors (P)
    Q = model.movie_factors.weight.data.cpu().numpy()  # Movie latent factors (Q)

    return P, Q

In [ ]:
best_params = {
    'n_factors': 10,  # Example best n_factors
    'alpha': 0.1,    # Example best alpha
    'lambda_reg': 0.1,  # Example best lambda_reg
    'n_epochs': 1000  # Example best n_epochs
}

In [ ]:
# Get the best P and Q matrices
P, Q = get_best_P_and_Q(ratings, best_params)

# Now you can use P and Q for recommendations or other tasks
print("Best P (user latent factors):")
print(P)

print("Best Q (movie latent factors):")
print(Q)

Best P (user latent factors):
[[ 1.8544238e-03  3.1837909e-03 -1.2574281e-03 ... -1.3407281e-03
  -2.3312890e-03 -5.8891056e-03]
 [ 1.5185139e-01  2.2354884e-01 -1.8456373e-01 ...  1.5082285e-01
   2.2259624e-01 -3.6205858e-02]
 [-5.0297003e-02  2.6915249e-01  1.5612359e-01 ...  7.0591420e-02
   1.9314998e-01 -1.0909347e-01]
 ...
 [ 9.9010405e-04  5.2981277e-04  2.0923044e-05 ... -8.5170445e-04
  -2.6103796e-04  2.3769599e-04]
 [ 8.5837342e-02 -3.2578591e-01 -1.9702701e-01 ...  4.2254277e-02
  -1.1068599e-01  6.0609568e-02]
 [ 1.5573979e-03  6.5696886e-04 -4.4141873e-04 ... -1.3986811e-04
   2.4958437e-03 -4.4661996e-04]]
Best Q (movie latent factors):
[[ 0.10072616 -0.07361765 -0.13247243 ... -0.02049093  0.22202346
   0.05959699]
 [ 0.36889273  0.55173945  0.9688095  ... -0.3188613  -0.15857975
  -0.43352142]
 [-0.46798006  0.06764051  0.43066737 ...  0.34535494  0.3340185
   0.02511735]
 ...
 [ 0.03553941  0.18098116 -0.6669928  ...  1.8792015   0.5931966
  -0.25338715]
 [-0.7530412

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to precompute movie similarities based on Q matrix
def precompute_movie_similarities(Q):
    """
    Precompute the cosine similarity between all movies using their latent vectors.
    Args:
    - Q (np.ndarray): The movie latent factors matrix of shape (num_movies, n_factors).

    Returns:
    - similarity_matrix (np.ndarray): Cosine similarity matrix of shape (num_movies, num_movies).
    """
    # Compute the cosine similarity between all pairs of movie latent vectors (rows of Q)
    similarity_matrix = cosine_similarity(Q)

    return similarity_matrix

In [ ]:
similarity_matrix = precompute_movie_similarities(Q)


In [ ]:
movie_index = movie_map_reverse['122904']

KeyError: '122904'

In [ ]:
similarities = similarity_matrix[1]
similarities

array([-0.3797824 ,  0.99999994,  0.30541682, ..., -0.26379767,
       -0.07518862,  0.19542608], dtype=float32)

In [ ]:
sorted_similarities = np.argsort(similarities)[::-1]
sorted_similarities

array([   1, 6644, 3747, ..., 9310, 8052, 4739])

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you've already trained your model and have best P and Q matrices
# P: User factors matrix (num_users, n_factors)
# Q: Movie factors matrix (num_movies, n_factors)

# Example of how you might load or have these matrices
# For example, loading P and Q from saved files if needed
# P = np.load('P.npy')   # User factors matrix
# Q = np.load('Q.npy')   # Movie factors matrix

# Here, we assume P and Q are already available
# Example: If you've trained them using PyTorch or another framework

# Precompute the movie similarity matrix based on the latent factor matrix Q
def precompute_movie_similarities(Q):
    """
    Precompute the cosine similarity between all movies using their latent vectors in Q.
    Args:
    - Q (np.ndarray): The movie latent factors matrix of shape (num_movies, n_factors).

    Returns:
    - similarity_matrix (np.ndarray): Cosine similarity matrix of shape (num_movies, num_movies).
    """
    similarity_matrix = cosine_similarity(Q)
    return similarity_matrix

# Function to get the movie ID from its title
def get_movie_id_from_title(title, movies_df):
    movie_id = movies_df[movies_df['title'] == title]['movieId'].values
    if len(movie_id) == 0:
        print(f"Movie title '{title}' not found in dataset.")
        return None
    return movie_id[0]

# Function to get top N recommendations based on movie similarity
def get_top_n_recommendations(movie_title, similarity_matrix, movie_map_reverse, top_n=20, movies_df=None):
    """
    Get the top N movie recommendations for a given movie title based on the precomputed similarity matrix.
    Args:
    - movie_title (str): The title of the movie for which recommendations are needed.
    - similarity_matrix (np.ndarray): The precomputed cosine similarity matrix.
    - movie_map_reverse (dict): Reverse mapping from movie index to movieId.
    - top_n (int): The number of recommendations to return.
    - movies_df (pd.DataFrame): DataFrame containing movie details, including movieId and title.

    Returns:
    - List of tuples: [(recommended_movie_id, similarity_score), ...]
    """
    movie_id = get_movie_id_from_title(movie_title, movies_df)
    if movie_id is None:
        return []

    # Ensure the movie_id is valid in the mapping
    if movie_id not in movie_map_reverse:
        print(f"Movie ID {movie_id} not found in movie_map_reverse.")
        return []

    # Get the index of the movie in the matrix
    movie_index = movie_map_reverse[movie_id]

    # Get the similarity scores for the given movie
    similarities = similarity_matrix[movie_index]

    # Sort the movies by similarity (descending order) and get the top N recommendations
    sorted_similarities = np.argsort(similarities)[::-1]

    # Exclude the movie itself from recommendations
    top_recommendations = []
    count = 0
    for idx in sorted_similarities:
        if idx != movie_index:  # Exclude the movie itself
            recommended_movie_id = movie_map_reverse[idx]
            similarity_score = similarities[idx]
            top_recommendations.append((recommended_movie_id, similarity_score))
            count += 1
        if count == top_n:
            break

    return top_recommendations

# Load your movies DataFrame (movies.csv) to map movie titles and ids
# movies = pd.read_csv('movies.csv')  # Make sure to have the movies.csv file

# Reverse movie mapping from movieId to index in Q (assuming Q corresponds to movieId order)
# movie_map_reverse = {idx: movie_id for idx, movie_id in enumerate(movies['movieId'].values)}

# Precompute the similarity matrix between all movies
similarity_matrix = precompute_movie_similarities(Q)

# Example usage: Get top 20 recommendations for a given movie title
movie_title = "Toy Story (1995)"  # Example movie title
top_recommendations = get_top_n_recommendations(movie_title, similarity_matrix, movie_map_reverse, top_n=20, movies_df=movies_df)

# Print top recommendations
print(f"Top recommendations for movie '{movie_title}':")
for rec_movie_id, score in top_recommendations:
    # Fetch movie title from movie_id
    rec_movie_title = movies_df[movies_df['movieId'] == rec_movie_id]['title'].values[0]
    print(f"Movie: {rec_movie_title}, Similarity: {score:.4f}")


Top recommendations for movie 'Toy Story (1995)':
Movie: Ask the Dust (2006), Similarity: 0.9025
Movie: Hollywood Chainsaw Hookers (1988), Similarity: 0.8929
Movie: Harrison Bergeron (1995), Similarity: 0.8677
Movie: 47 Ronin (2013), Similarity: 0.8456
Movie: Yankee Doodle Dandy (1942), Similarity: 0.8415
Movie: AVP: Alien vs. Predator (2004), Similarity: 0.8346
Movie: Secret of NIMH, The (1982), Similarity: 0.8285
Movie: Prometheus (2012), Similarity: 0.8265
Movie: Last Supper, The (1995), Similarity: 0.8236
Movie: Pawn (2013), Similarity: 0.8215
Movie: Short Circuit 2 (1988), Similarity: 0.8143
Movie: Simple Wish, A (1997), Similarity: 0.8124
Movie: Tremors II: Aftershocks (1996), Similarity: 0.8105
Movie: Now You See Me 2 (2016), Similarity: 0.8103
Movie: Giant Spider Invasion, The (1975), Similarity: 0.8092
Movie: Little Princess, A (1995), Similarity: 0.8072
Movie: Bachelor, The (1999), Similarity: 0.8064
Movie: In China They Eat Dogs (I Kina spiser de hunde) (1999), Similarity: 0

Variables saved to 'saved_variables.pkl'
